In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../Data-20241219T000653Z-001/Data/Copy of Week2_challenge_data_source(CSV).csv')

In [ ]:
df.head()

In [ ]:
print("\nDataset Info:")
df.info()

In [ ]:
print("\nBasic statistics:")
df.describe()

In [ ]:
# Handle missing value by replacing with column mean
df.isnull().sum()

In [ ]:
missing_percentage = (df.isnull().sum() / len(df)) * 100
print(missing_percentage.sort_values(ascending=False))


In [ ]:
# Step 1: Filter numeric columns
numeric_columns = df.select_dtypes(include=['number']).columns

# Step 2: Filter numeric columns with missing values
numeric_columns_with_missing = [col for col in numeric_columns if df[col].isnull().sum() > 0]

# Step 3: Replace missing values with the mean
for col in numeric_columns_with_missing:
    mean_value = df[col].mean()  # Calculate the mean for the column
    df[col].fillna(mean_value, inplace=True)  # Replace missing values with the mean

# Validate
print(df.isnull().sum())  # Ensure no missing values remain in numeric columns


In [ ]:


# Aggregating per user
result = df.groupby('IMSI').agg(
    number_of_xDR_sessions=('Dur. (ms)', 'size'),  # Count of sessions
    total_session_duration=('Dur. (ms)', 'sum'),  # Sum of session durations
    total_download=('Total DL (Bytes)', 'sum'),  # Sum of download volumes
    total_upload=('Total UL (Bytes)', 'sum'),  # Sum of upload volumes
    total_data_volume=('Total DL (Bytes)', lambda x: x.sum() + df.loc[x.index, 'Total UL (Bytes)'].sum())  # Total data volume
).reset_index()

# Output the result
print(result)


In [ ]:
# Check data types and summarize dataset
print(df.info())  # Lists column names and data types
print(df.describe(include='all'))  # Summarizes numerical and categorical data

# Identifying missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)


In [ ]:
# Basic metrics for numerical columns
metrics = df.describe()
print(metrics)


In [ ]:
# Compute dispersion parameters
dispersion = df.var(), df.std()
print("Variance and Standard Deviation:\n", dispersion)
